#### Calling RunPod Endpoint

In [3]:
# Using the same BASE_URL you defined
INTERNAL_PORT = 30000 # Need to expose port from runpod instance editting window 
POD_ID = "jyql518x9mlrdc" # Specific to each runpod instance
BASE_URL = f"https://{POD_ID}-{INTERNAL_PORT}.proxy.runpod.net" # replace "localhost" with this

import requests

prompts = [  
    "What is the capital of France?",
    "Tell me a short joke"
] * 20

grammar_str = """
root ::= "Batman: " letter letter letter letter letter
letter ::= [a-zA-Z]
"""

request_data = {
    "prompts": prompts,
    "grammar": grammar_str
}

# response with grammar constraint
response = requests.post(
    f"{BASE_URL}/generate_cfg",
    json=request_data,
)

# response without grammar constraint 
# response = requests.post(
#     f"{BASE_URL}/generate",
#     json = prompts
# )


if response.status_code == 200:
    results = response.json()
    print("\nResults:")
    print(results)
else:
    print(f"Error: {response.status_code}")
    print(response.text)
    
    
# Next question is how to convert interleaved text & image data into prompt
# llama3.2-vision-instruct should provide such information right? 



Results:
{'results': ['Batman: Paris', 'Batman: Ibito', 'Batman: Paris', 'Batman: IddnA', 'Batman: Paris', 'Batman: Whywk', 'Batman: Ithin', 'Batman: Iaals', 'Batman: Theak', 'Batman: Ithyw', 'Batman: Their', 'Batman: IUPUI', 'Batman: Paris', 'Batman: Icing', 'Batman: paris', 'Batman: Iaind', 'Batman: Idaeh', 'Batman: Whyul', 'Batman: Paris', 'Batman: whyRA', 'Batman: TheaJ', 'Batman: Ikaru', 'Batman: Theal', 'Batman: WhyIU', 'Batman: Paris', 'Batman: WhyIU', 'Batman: Ident', 'Batman: WhyTD', 'Batman: Noted', 'Batman: Whatd', 'Batman: Theur', 'Batman: Whyhq', 'Batman: Theth', 'Batman: Ieauc', 'Batman: Tssss', 'Batman: WhydI', 'Batman: Thata', 'Batman: Ivard', 'Batman: Thean', 'Batman: WhyVy']}
